## Limpieza Inicial de Datos, Unión de Tablas y Formateo de Fechas

Como parte del proceso de la generación de los modelos de pronóstico de contamianción, es necesario realizar un conjunto de pasos a los datos obtenidos de la página de [Monitoreo de Calidad del Aire de la Ciudad de México](http://www.aire.cdmx.gob.mx/default.php), como se mencionó en el archivo de metodología, los datos a usar son los datos verificados por los expertos de la SEDEMA, los datos para meteorología y contaminanción se pueden obtener acontinuación:

- [Meteorología](http://www.aire.cdmx.gob.mx/default.php?opc='aKBhnmI='&opcion=Zw==)
- [Contamianción](http://www.aire.cdmx.gob.mx/default.php?opc='aKBhnmI='&opcion=Zg==)


En este notebook vamos a juntar los archivos de contaminnación y meoteorología de cada año en un solo archivo, y también vamos a generar columnas de informaciónn temporal para cada mediciónn y por úlitmo se va a obtener el promedio móvil de el PM10 y PM2.5. 

- __Datos recibidos:__ Limpieza de datos inicial
- __Responsable:__ Daniel Bustillos
- __Contacto:__  juandaniel.bucam@gmail.com

__Pasos y descripción general del notebook__


Una vez que se tienen los datos limpios, es necesario aplicar una serie de manipulaciones:


1. 


2.  


3. 


4. 


5. 



Juntaremos los dataframes con una PivotTable y las agruparemos por el momento de la medición

In [1]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
import pandas as pd
import matplotlib
import seaborn as sns

from datetime import timedelta

#### Definimos tres funciones para formatear el formato de las fechas:

Convertir el formato de 1 a 24 horas al formato de 0 a 23 horas. Por defecto python trabaja con el formato de 0 a 23 horas, es conveniente trabajar en este formato debido a que muchas de las funciones implementadas en python u otras librerias suponen que este es el formato de las fechas.

Como referencia, el formato original de las fechas, es DD/MM/AAAA hh:mm

In [2]:
def time_converter(x): 
    x0a = x.split(" ")[0] #separamos la fecha y la hora por el espacio en blanco que las separa
    x0 = x0.split("/") #separamos el día mes y año  por la diagonal entre ellas
    x1 = x.split(" ")[1] # nos quedamos con la hora
    if x1[:].endswith("24:00"):
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" 00:00" # si trmina en 24 la 
    else:
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" "+ x1[:]

In [3]:
def time_converter_guion(x):
    x0 = x.split(" ")[0]
    x0 = x0.split("-")
    x1 = x.split(" ")[1]
    if x1[:].endswith("24:00"):
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" 00:00"
    else:
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" "+ x1[:]

In [4]:
def time_converter_date(x):
    x0 = x.split("/")
    return x0[2]+"-"+x0[1]+"-"+x0[0]

## Carga de los conjuntos de datos   <a class="anchor" id="limpieza-bullet"></a>

Definamos el año a limpiar:

In [7]:
a="2018"

### Metereología

In [8]:
met_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/Metereología/meteorología_' + a + ".CSV"),header=10)
met_2018 = met_2018.dropna(how='any')
met_2018 = met_2018.drop(['unit'], axis=1)

FileNotFoundError: [Errno 2] File b'/Users/danielbustillos/Documents/servicio/Contaminacio\xcc\x81n PM10/Metereologi\xcc\x81a/meteorologi\xcc\x81a_2018.CSV' does not exist: b'/Users/danielbustillos/Documents/servicio/Contaminacio\xcc\x81n PM10/Metereologi\xcc\x81a/meteorologi\xcc\x81a_2018.CSV'

In [1633]:
#met_2018.head(20)

### Contaminantes

Una vez que se descargado los archivos de contaminación y meteorología, se lee uno por uno y se deshechan las entradas vacías, es importante notar que se tiene una columna de fechas, una columna para indicar el atributo y una columna conn su medición:

In [ ]:
cont_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/Contaminantes/contaminantes_'+ a +'.CSV'),header=10)
cont_2018 = cont_2018.dropna(how='any')
cont_2018 = cont_2018.drop(['unit'], axis=1)

In [ ]:
cont_2018

In [1635]:
#cont_2018.head(200)

In [1636]:
radA_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/UV/UVA/UVA_'+ a +'.csv'),header=8)
radA_2018 = radA_2018.fillna(0)
radA_2018 = radA_2018.drop(['unit'], axis=1)

### Radiación UVB

In [1637]:
radB_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/UV/UVB/UVB_'+ a +'.csv'),header=8)
radB_2018 = radB_2018.fillna(0)
radB_2018 = radB_2018.drop(['unit'], axis=1)

## Append Radiación

Juntamos los df. de UVA y UVBm

In [1638]:
rad_2018 = radA_2018.append(radB_2018, ignore_index=True)

# Limpieza paso a paso

### Pres

Nos quedamos con las columnas indicadas y reseteamos el index:

In [1639]:
pre_ACO = pre_2018
pre_ACO = pre_ACO.reset_index(drop=False)
pre_ACO = pre_ACO[["Date","cve_station","parameter","value"]]

In [1640]:
pre_ACO = pre_ACO.rename(columns={'Date': 'date', 'cve_station': 'id_station','parameter': 'id_parameter'})

### Contaminantes

In [1641]:
cont_2018.head()

,date,id_station,id_parameter,value
5,01/01/2019 01:00,ACO,PM10,139.0
14,01/01/2019 01:00,ATI,PM10,92.0
21,01/01/2019 01:00,CAM,PM10,122.0
22,01/01/2019 01:00,CAM,PM2.5,90.0
23,01/01/2019 01:00,CAM,PMCO,32.0


In [1642]:
cont_ACO = cont_2018
#cont_ACO = cont_ACO[(cont_ACO["id_parameter"] == "PM10")]
cont_ACO = cont_ACO.reset_index(drop=False)
cont_ACO = cont_ACO[["date","id_station","id_parameter","value"]]

### Metereologia

In [1643]:
met_ACO = met_2018
#cont_ACO = cont_ACO[(cont_ACO["id_parameter"] == "PM10")]
met_ACO = met_ACO.reset_index(drop=False)
met_ACO = met_ACO[["date","id_station","id_parameter","value"]]

### Radiación

In [1644]:
rad_2018 = rad_2018.rename(columns={'Date': 'date', 'cve_station': 'id_station','parameter': 'id_parameter'})

In [1645]:
rad_ACO = rad_2018
#cont_ACO = cont_ACO[(cont_ACO["id_parameter"] == "PM10")]
rad_ACO = rad_ACO.reset_index(drop=False)
rad_ACO = rad_ACO[["date","id_station","id_parameter","value"]]

In [1646]:
rad_ACO.head()

,date,id_station,id_parameter,value
0,01-01-2019 01:00,CHO,UVA,0.0
1,01-01-2019 01:00,CUT,UVA,0.0
2,01-01-2019 01:00,FAC,UVA,0.0
3,01-01-2019 01:00,LAA,UVA,0.0
4,01-01-2019 01:00,MER,UVA,0.0


## Pivot_Table <a class="anchor" id="pivot-bullet"></a>

### Pre

Creamos la tabla dinámica o pivote:

In [1647]:
pre_ACO_hour = pd.pivot_table(pre_ACO,index=["date","id_station"],columns=["id_parameter"])

Reseteamos el index para desacernos del multiindex:

In [1648]:
pre_ACO_hour = pre_ACO_hour.reset_index(drop=False)

In [1649]:
pre_ACO_hour.columns = pre_ACO_hour.columns.droplevel()

Reefinimos las columnas:

In [1650]:
pre_ACO_hour["id_station"] = pre_ACO_hour.iloc[:,1]
pre_ACO_hour["date"] = pre_ACO_hour.iloc[:,0]

Nos deshacemos de las columnas sin nombre:

In [1651]:
pre_ACO_hour = pre_ACO_hour.drop([""],axis=1)

Aplicamos las funciones definidas al principio del documento:

In [1]:
pre_ACO_hour['date'] = pre_ACO_hour.apply(lambda rojw: time_converter_guion(rojw['date']), axis=1) 

NameError: name 'pre_ACO_hour' is not defined

Convertimos lacolumna fecha en datetime:

In [1653]:
pre_ACO_hour['date'] =  pd.to_datetime(pre_ACO_hour['date'], format='%Y-%m-%d %H:%M')

In [1654]:
pre_ACO_hour = pre_ACO_hour[[ "date" ,"id_station",'PA']]

In [1655]:
#pre_ACO_hour = pre_ACO_hour.groupby(["date"]).mean().reset_index(drop=False)

### Contaminantes

In [1656]:
cont_ACO_hour = pd.pivot_table(cont_ACO,index=["date","id_station"],columns=["id_parameter"])
cont_ACO_hour = cont_ACO_hour.reset_index(drop=False)
cont_ACO_hour.columns = cont_ACO_hour.columns.droplevel()
cont_ACO_hour["id_station"] = cont_ACO_hour.iloc[:,1]
cont_ACO_hour["date"] = cont_ACO_hour.iloc[:,0]
cont_ACO_hour = cont_ACO_hour.drop([""],axis=1)
cont_ACO_hour['date'] = cont_ACO_hour.apply(lambda row: time_converter(row['date']), axis=1) 
cont_ACO_hour['date'] =  pd.to_datetime(cont_ACO_hour['date'], format='%Y-%m-%d %H:%M')
cont_ACO_hour = cont_ACO_hour[[ "date" ,"id_station",'CO', 'NO', 'NO2', 'NOX', 'O3', 'PM2.5', 'PMCO', 'SO2','PM10']]

In [1657]:
#cont_ACO_hour['date'] = cont_ACO_hour["date"].str[0:10]

In [1658]:
cont_ACO_hour.head()

id_parameter,date,id_station,CO,NO,NO2,NOX,O3,PM2.5,PMCO,SO2,PM10
0,2019-01-01 01:00:00,ACO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.0
1,2019-01-01 01:00:00,AJM,NaN,NaN,NaN,NaN,NaN,23.0,19.0,NaN,42.0
2,2019-01-01 01:00:00,ATI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.0
3,2019-01-01 01:00:00,BJU,NaN,NaN,NaN,NaN,NaN,47.0,17.0,NaN,64.0
4,2019-01-01 01:00:00,CAM,NaN,NaN,NaN,NaN,NaN,90.0,32.0,NaN,122.0


In [1659]:
cont_ACO_hour_gb = cont_ACO_hour.groupby(['date'])['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM2.5', 'PMCO',
       'SO2', 'PM10'].max()
cont_ACO_hour_gb.reset_index(inplace=True)

### Metereología

In [1660]:
met_ACO_hour = pd.pivot_table(met_ACO,index=["date","id_station"],columns=["id_parameter"])
met_ACO_hour = met_ACO_hour.reset_index(drop=False)
met_ACO_hour.columns = met_ACO_hour.columns.droplevel()
met_ACO_hour["id_station"] = met_ACO_hour.iloc[:,1]
met_ACO_hour["date"] = met_ACO_hour.iloc[:,0]
met_ACO_hour = met_ACO_hour.drop([""],axis=1)
met_ACO_hour['date'] = met_ACO_hour.apply(lambda row: time_converter(row['date']), axis=1) 
#met_ACO_hour['date'] =  pd.to_datetime(met_ACO_hour['date'], format='%Y-%m-%d %H:%M')
met_ACO_hour = met_ACO_hour[["date","id_station","RH","TMP","WSP","WDR"]]

### Radiación

In [1661]:
rad_ACO.head()

,date,id_station,id_parameter,value
0,01-01-2019 01:00,CHO,UVA,0.0
1,01-01-2019 01:00,CUT,UVA,0.0
2,01-01-2019 01:00,FAC,UVA,0.0
3,01-01-2019 01:00,LAA,UVA,0.0
4,01-01-2019 01:00,MER,UVA,0.0


In [1662]:
rad_ACO = pd.pivot_table(rad_ACO,index=["date","id_station"],columns=["id_parameter"])
rad_ACO = rad_ACO.reset_index(drop=False)
rad_ACO.columns = rad_ACO.columns.droplevel()
rad_ACO["id_station"] = rad_ACO.iloc[:,1]
rad_ACO["date"] = rad_ACO.iloc[:,0]
rad_ACO = rad_ACO.drop([""],axis=1)
rad_ACO['date'] = rad_ACO.apply(lambda row: time_converter_guion(row['date']), axis=1)
rad_ACO['date'] =  pd.to_datetime(rad_ACO['date'], å
rad_ACO = rad_ACO[[ "date","id_station",'UVA',"UVB"]]
rad_ACO = rad_ACO.dropna(how='any')

# Merge de Dataframes   <a class="anchor" id="merge-bullet"></a>

Por hora: no es posible añadir precipitación ya que de origen viene por día.

Juntamos los dataframes:

In [1663]:
cont_ACO_hour['date'] =  pd.to_datetime(cont_ACO_hour['date'], format='%Y-%m-%d %H:%M')
met_ACO_hour['date'] =  pd.to_datetime(met_ACO_hour['date'], format='%Y-%m-%d %H:%M')

In [1664]:
data_hour_merge = pd.merge(cont_ACO_hour, met_ACO_hour, on=["date","id_station"],how="outer")

In [1665]:
data_hour_merge = pd.merge(data_hour_merge, pre_ACO_hour, on=["date","id_station"],how="outer")

In [1666]:
data_hour_merge = pd.merge(data_hour_merge, rad_ACO, on=["date","id_station"],how="outer")

Eliminamos los NAN:

Definimos las columnas hora, dia, mes

Cambiamos el nombre de date a fecha:

In [1667]:
data_hour_merge = data_hour_merge.rename(columns={'date': 'fecha'})

In [1668]:
#data_hour_merge = data_hour_merge.drop(['date'],axis=0)

Exportamos:

In [1669]:
data_hour_merge = data_hour_merge.dropna()

In [1670]:
data_hour_merge = data_hour_merge.dropna(subset=['PM10'])

In [1671]:
data_hour_merge['hora'] = data_hour_merge['fecha'].astype(str).str[10:13]

In [1672]:
data_hour_merge['dia'] = data_hour_merge['fecha'].astype(str).str[8:10]

In [1673]:
data_hour_merge['mes'] = data_hour_merge['fecha'].astype(str).str[5:7]

In [1674]:
data_hour_merge['fecha'] = data_hour_merge['fecha'].astype(str).str[0:10]

In [1675]:
data_hour_merge.head()

id_parameter,fecha,id_station,CO,NO,NO2,NOX,O3,PM2.5,PMCO,SO2,...,RH,TMP,WSP,WDR,PA,UVA,UVB,hora,dia,mes
94,2019-01-01,MER,1.3,43.0,36.0,80.0,1.0,82.0,36.0,5.0,...,69.0,11.0,0.8,130.0,584.1,0.0,0.0,04,01,01
103,2019-01-01,TLA,1.2,41.0,30.0,71.0,1.0,59.0,21.0,3.0,...,66.0,9.8,0.7,249.0,581.0,0.0,0.0,04,01,01
126,2019-01-01,MER,1.5,57.0,39.0,96.0,2.0,110.0,43.0,6.0,...,68.0,10.0,1.0,82.0,584.0,0.0,0.0,05,01,01
135,2019-01-01,TLA,0.8,20.0,30.0,50.0,1.0,58.0,26.0,3.0,...,66.0,9.1,0.5,214.0,581.0,0.0,0.0,05,01,01
158,2019-01-01,MER,1.4,55.0,36.0,91.0,2.0,165.0,62.0,6.0,...,70.0,9.5,0.5,69.0,584.0,0.0,0.0,06,01,01


In [1676]:
data_hour_merge.to_csv(str("/Users/danielbustillos/Documents/servicio/Contaminación PM10/Outputs/por_hora/cont_hora" + a + ".csv"))